# Time series classification

In [ ]:
import numpy as np
from time import sleep 

# import our modules
from utils import *
from train import *
from MultiLayerPerceptron import *
from ConvNet import *

datasets = np.loadtxt('datasets.txt',dtype=str)
# download_datasets(datasets)  # uncomment this to download the data
dataset_dictionary = data_dictionary(datasets)

In [ ]:
# training loop
epochs = 1000
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device: {}\n'.format(device))

for dataset,dataloader in dataset_dictionary.items():

    # setting up
    print_dataset_info(dataset,dataloader)
    sleep(1)
    
    time_steps = dataloader['test'].dataset.x.shape[-1]
    n_classes  = len(np.unique(dataloader['test'].dataset.y))
    
    learning_rate = 0.0001
    
    # MLP
    model = MultiLayerPerceptron(time_steps,n_classes)
    if torch.cuda.device_count() > 0:
        model = nn.DataParallel(model)
    model.to(device)
    model, history = train(dataloader['train'],
                           dataloader['test'],
                           device, model,
                           epochs, learning_rate,
                           save=False)
    print('MPCE: {0:.4f}'.format(mpce(model,dataloader['test'],device)))
    sleep(1)
    
    # ConvNet
    model = ConvNet(time_steps,n_classes)
    if torch.cuda.device_count() > 0:
        model = nn.DataParallel(model)
    model.to(device)

    model, history = train(dataloader['train'],
                           dataloader['test'],
                           device, model,
                           epochs, learning_rate,
                           save=False)
    print('MPCE: {0:.4f}'.format(mpce(model,dataloader['test'],device)))
    print()